In [1]:
import numpy as np
import time
import datetime
from pathlib import Path

In [2]:
from simulations.distributions import CPIDM

In [3]:
rate = 2
entropy = 42
concentration = 1.0
number_samples = int(15e6)

In [4]:
seed = np.random.SeedSequence(entropy)

In [5]:
base_relative_abundances = [1e-4, 1e-3, 1e-2]

relative_abundances = [relative_abundance * number
                       for relative_abundance 
                       in base_relative_abundances
                       for number in (1,2,5) 
                       for repeat in range(10)]

relative_abundances += [1-sum(relative_abundances)]
frequencies = np.array(relative_abundances)

In [6]:
rng = np.random.default_rng(seed)
interaction_matrix = 2*(rng.random(size=(frequencies.size, frequencies.size)) - 0.5)
# https://numpy.org/devdocs/reference/random/generated/numpy.random.Generator.integers.html
interaction_matrix *= rng.integers(low=0, high=2, size=(frequencies.size, frequencies.size))

## CPIDM - Compound Poisson Interaction Dirichlet-Multinomial (concentration 1)

In [7]:
def CPIDM_simulation(concentration, frequencies, interaction_matrix, rate, 
                      seed, number_samples):
    rng = np.random.default_rng(seed)

    results = [CPIDM(concentration, frequencies, interaction_matrix, rate, rng) for sample in range(number_samples)]
    
    results = [result.reshape((1,-1)) for result in results]
    results = np.concatenate(results, axis=0)
    # shape of results is now (number_droplets, number_strains)
    return results

In [8]:
results_filename = 'CPIDM_results.npz'
results_file = Path(results_filename)

if results_file.is_file():
    # simulation already ran successfully on previous attempt
    pass
else:
    start_time = time.time()
    results = CPIDM_simulation(concentration=concentration,
                    interaction_matrix=interaction_matrix,
                    rate=rate, seed=seed, 
                    number_samples=number_samples,
                    frequencies=frequencies)
    runtime = time.time() - start_time
    
    with open('runtime.log', 'a') as file_pointer:
        # https://stackoverflow.com/a/775095/10634604
        runtime_string = str(datetime.timedelta(seconds=runtime))
        file_pointer.write('\nRuntime was {} in Hours:Minutes:Seconds.\n'.format(runtime_string))

    np.savez_compressed(results_filename, droplets=results)